# Bilden eines Models mit Original und einem Filter

## Import der notwendigen Pakete

In [1]:
# Import Tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Import des Datasets

Zusätzlich zum Laden des Datasets (2 Klassen) wird hier bereits durch den ImageDataGenerator ein Preprocessing durchgeführt / vorbereitet.
Zunächst wird versucht eine Klassifizierung durchzuführen, bei dem die Bilder in Graustufen geladen werden.

In [2]:
# path to dataset
directory = "./FACD_image"
# choosen classes (origin + filter) for classification
classes = ["Origin", "Amaro"];
target_size = (400,400)
seed = 42;

# create a image generator for keras, that can load images batchwise
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_iterator = tf.keras.preprocessing.image.DirectoryIterator(
    directory, data_generator, target_size=target_size, color_mode='grayscale',
    classes=classes, class_mode='categorical', batch_size=32, shuffle=True, seed=seed,
    follow_links=False, subset='training', interpolation='nearest', dtype=None
)

test_iterator = tf.keras.preprocessing.image.DirectoryIterator(
    directory, data_generator, target_size=target_size, color_mode='grayscale',
    classes=classes, class_mode='categorical', batch_size=32, shuffle=True, seed=seed,
    follow_links=False, subset='validation', interpolation='nearest', dtype=None
)

Found 2048 images belonging to 2 classes.
Found 512 images belonging to 2 classes.


In [3]:
## Building the CNN-Model
input_shape = (400,400,1)
num_classes = 2

model = tf.keras.Sequential()
model.add(keras.Input(shape=input_shape))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation="softmax"))
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 398, 398, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 199, 199, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 197, 197, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 614656)            0         
_________________________________________________________________
dropout (Dropout)            (None, 614656)            0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1

In [4]:
batch_size = 32
epochs = 10
steps_per_epoch = 2048 / 32


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(
    x=train_iterator, y=None, epochs=epochs, verbose=1, callbacks=None,
    validation_data=test_iterator, initial_epoch=0, steps_per_epoch=steps_per_epoch, validation_freq=1,
    max_queue_size=10
)

C:\Users\Martin\Anaconda3\envs\dhbw20\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Martin\Anaconda3\envs\dhbw20\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 64.0 steps, validate for 16 steps
Epoch 1/10


C:\Users\Martin\Anaconda3\envs\dhbw20\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Martin\Anaconda3\envs\dhbw20\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


12/64 [====>.........................] - ETA: 1:46 - loss: 487.4996 - accuracy: 0.6108

KeyboardInterrupt: 

In [ ]:
filepath = "./first_model.hdf5"

model.save(
   filepath, overwrite=True, include_optimizer=True, save_format="h5",
    signatures=None, options=None
)

In [ ]:
history.history.get("loss")